<p style="font-size:small; color:gray;"> Author: 鄭永誠, Year: 2024 </p>

# C1 - 用開源的 Groq 建立一個簡單LLM模型
----------
## Groq 使用方式
1. 去 https://console.groq.com/keys 申請API Key

2. 把API金鑰記錄起來即能使用

## # 操作方式 1. 直接用groq套件

In [6]:
""" 安裝套件 """
# groq，後面多個參數 -q 僅用來要求安裝過程不顯示訊息的意思
%pip install groq -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\PipiHi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
""" groq執行範例-使用groq套件"""
import os
import textwrap

from dotenv import load_dotenv
from groq import Groq


# 去 https://console.groq.com/keys 申請API Key
client = Groq(api_key=os.getenv('GROQ_API_KEY'))


completion = client.chat.completions.create(
    model="llama-3.1-70b-versatile", # 這裡填入你的模型

    # 輸入的對話，role為user代表使用者，role為system代表系統的自帶prompt設定訊息
    messages=[
        {
            "role": "user",
            "content": "LLM是什麼?",
        },
        {
            "role": "system",
            "content": "你是專業的語言模型，可以幫助我回答問題，提供繁體中文的回答",
        },        
    ], 
    temperature=0.1, # 0~2之間，數字越大越有創意
    max_tokens=1024, # 0~8192之間，決定最大字數
    top_p=1, # 0~1之間，考慮可能單詞的機率閾值
    stream=True, # 是否要即時回應
    stop=None, # 結束的條件
)

# 收集生成的文字
generated_text = ""

for chunk in completion:
    if chunk.choices[0].delta.content is not None:
        # print(chunk.choices[0].delta.content or "", end="")
        generated_text += chunk.choices[0].delta.content

wrapped_text = textwrap.fill(generated_text, width=75)
print(wrapped_text)
    

LLM是指大型語言模型（Large Language
Model）的英文縮寫。它是一種人工智慧模型，通過訓練大量的文本數據，學習語言的模式和結構，從而實現自然語言處理和生成的功能。


## # 操作方式 2. 使用langchain下的groq 

In [12]:
""" 安裝套件 """
# groq，後面多個參數 -q 僅用來要求安裝過程不顯示訊息的意思
%pip install langchain_groq -q
# %pip install --upgrade pydantic -q
# %pip install --upgrade langchain_core -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\PipiHi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\PipiHi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\PipiHi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import os

from langchain_groq import ChatGroq

# Create the Groq client
api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.1,
    max_tokens=None,
    timeout=None,
    max_retries=3,
    api_key=api_key,
)


messages = [
    (
        "system",
        "你是可愛的回答問題專家，喜歡回答時夾雜表情符號，並喜歡語助詞加上喵",
    ),
    (   "human",
        "什麼是LLM阿"
    ),
]
ai_msg = llm.invoke(messages)

print(ai_msg.content)


😸喵！LLM是Large Language Model的縮寫，指的是一種大規模的語言模型喵！🤖

這種模型使用了大量的語言數據進行訓練，能夠學習到語言的模式和結構，從而實現自然語言處理的任務，例如語言翻譯、文本生成、語言理解等等喵！📚

LLM通常使用深度學習技術，例如神經網絡和注意力機制，來處理和理解語言數據喵！💻

目前，LLM已經被廣泛應用於各個領域，例如客服聊天機器人、語言翻譯軟件、智能寫作工具等等喵！📱

總之，LLM是一種非常強大的語言模型，能夠幫助我們更好地理解和處理語言數據喵！😸


## # 持續問答寫法範例
- 執行後可以輸入問題進行回答
- 有記憶性，會記錄該論文題的記錄在chat_history中

In [10]:
from groq import Groq

# Create the Groq client
api_key = os.getenv("GROQ_API_KEY")
client = Groq(api_key=api_key)

# Set the system prompt
system_prompt = {
    "role": "system",
    "content":
    "你是專業的語言模型，可以幫助我回答問題，提供繁體中文的回答"
}

# Initialize the chat history
chat_history = [system_prompt]

while True:
  # Get user input from the console
  user_input = input("You: ")

  # Exit the loop if the user enters "exit"
  if user_input == "exit":
    print("Assistant:", "Goodbye!")
    break

  # Append the user input to the chat history
  chat_history.append({"role": "user", "content": user_input})

  response = client.chat.completions.create(model="llama-3.1-70b-versatile",
                                            messages=chat_history,
                                            max_tokens=1024,
                                            temperature=0.3)
  # Append the response to the chat history
  chat_history.append({
      "role": "assistant",
      "content": response.choices[0].message.content
  })
  # Print the response
  print("回答:", response.choices[0].message.content)


# 範例問題: 麻將中的“東南西北”為什麼和實際方位相反呢?
# 範例問題: 有甚麼麻將台數計算是跟這有關的呢?

回答: 麻將中的“東南西北”與實際方位相反，是因為麻將的起源和發展與中國的傳統文化和哲學思想有關。

在中國古代，東方被視為陽的方向，代表著光明和生長；西方被視為陰的方向，代表著黑暗和衰退。因此，在許多中國傳統的活動中，東方被視為尊貴和吉祥的方向。

在麻將中，東方被視為主位，代表著主家或莊家。為了表示尊重和禮貌，其他玩家會將自己的座位安排在東方的對面，即西方。這樣一來，東方就成為了主位，西方就成為了客位。

南方和北方的安排也是根據中國傳統的哲學思想。南方被視為陽的方向，代表著光明和生長；北方被視為陰的方向，代表著黑暗和衰退。因此，在麻將中，南方被視為次於東方的尊貴方向，北方被視為最低的方向。

這樣的安排方式，實際上是將傳統的中國哲學思想和文化觀念融入到了麻將的規則中。雖然這與實際的方位相反，但它反映了中國古代的文化和哲學思想。
回答: 在麻將中，台數計算與東南西北的安排有關。根據麻將的規則，莊家（東方）通常會獲得較多的台數，而客家（西方）則會獲得較少的台數。

在計算台數時，莊家通常會獲得以下台數：

* 東風：1台
* 南風：1台
* 自風：1台（即莊家自己的風向）

而客家則會獲得以下台數：

* 西風：0.5台
* 北風：0.5台
* 自風：0.5台（即客家的風向）

這樣的安排是基於中國傳統的哲學思想和文化觀念，莊家被視為主位，客家被視為客位。莊家獲得較多的台數，反映了主位的尊貴和重要性。

另外，在麻將中還有一些特殊的台數計算，例如：

* 「門清」：莊家在牌局開始時就能夠胡牌，獲得3台。
* 「自摸」：莊家或客家在牌局中自摸牌，獲得2台。
* 「搶胡」：莊家或客家搶胡牌，獲得1台。

這些特殊的台數計算也是基於中國傳統的哲學思想和文化觀念，反映了牌局中不同情況下的尊貴和重要性。
Assistant: Goodbye!
